In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ipywidgets as wg
from networkx.drawing.nx_pydot import graphviz_layout

# Ising model function definitions

In [2]:
def energy_calc(lat):
    ''' Calculated total energy of the lattice.
    
        Parameters:
            lat: [N] numpy array containing only 1 and -1 values representing the spin configuration.
        
        Returns: dtype float
            Energy of the spin configuration'''
    
    a = lat[:N-1]*lat[1:]
    # To ensure periodic boundary conditions (or to have the chain) the last entry is multiplied with the first entry of the lattice array. 
    b = lat[0]*lat[N-1]
    # Energy:
    E = -1*(np.sum(a)+np.sum(b))
    return(E)

In [3]:
def energy_nn(lat,x):
    ''' Calculate nearest neighbour interaction energy of 1 spin in the lattice.
    
        Parameters:
            lat: dtype numpy array
                [N] numpy array containing only 1 and -1 values representing the spin configuration.
            
            x: dtype int
                The coordinate of the spin of interest
        
        Returns: dtype float
             nearest neighbour interaction energy of the selected spin '''
    
    s = lat[x]
    nn = lat[(x+1)%N] + lat[(x-1)%N] 
    E = -1*nn*s
    return(E)

In [4]:
def specific_heat(x):
    ''' Calculates specific heat.
        
        Parameters: 
            x: dtype numpy array
                One dimensional array containing the energy data (or equivalent quantity)
            
        Returns: dtype numpy array
            One dimensional array containing the specific heat data'''
    
    cv = (np.var(x))/(T**2)
    return(cv)

In [5]:
def susceptiblity(mag):
    ''' Calculates specific heat.
        
        Parameters: 
            mag: dtype numpy array
                One dimensional array containing the magnetization data
            
        Returns: dtype numpy array
            One dimensional array containing the susceptibility data'''
    
    susc = (np.var(mag))/T
    return(susc)

In [6]:
def metropolis(lat,E,M):
    ''' Implement metropolis algorithm for a single spin flip.
    
        Parameters: 
            lat: dtype numpy array
                [N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    # Select random spin (propose trial lattice with probability 1/L):
    x = np.random.randint(0,N)
    trial_lat = np.copy(lat)
    trial_lat[x] *= -1
    # Calculate energy difference between the old and trial configurations:
    dE = energy_nn(trial_lat,x) - energy_nn(lat,x)
    # Implement second stage of the metropolis algorithm:
    if dE <= 0:
        E = E + dE
        M = M - lat[x] + trial_lat[x] # substract old spin value and add new spin value to the magnetisation
        lat[x] = trial_lat[x]
    else:
        r = np.random.uniform(0,1)
        #p=np.exp(-dE/tau)
        p = exp[0]
        if r < p:
            E = E + dE
            M = M - lat[x] + trial_lat[x] # substract old spin value and add new spin value to the magnetisation
            lat[x] = trial_lat[x]
    return(lat,E,M)

In [7]:
def Wolffbacktrack(x,cluster,visited):
    ''' Part of the Wolff algorithm, it checks whether a spin is already in the cluster'''
    if cluster[x]:
        return False
    if visited[x]:
        return True#False
    return True

def grow_cluster(lattice,x,ClusterSpin,E,M, cluster,visited):
    """ Starting from a randomly chosen spin, creates a cluster with the Wolff Algorithm.
        
        Parameters:
            lattice: dtype numpy array
                The [N] array representing the spin configuration
            
            x: dtype int
                The coordinate of the spin that is added to the cluster
                
            ClusterSpin: dtype int
                The spin value of the cluster
            
            E and M: dtype int or float
                The energy and magnetization of the lattice
            
            cluster: dtype boolean numpy array
                [N] boolean numpy array indicating which spins are in the cluster
                
            visited: dtype boolean numpy array
                [N] boolean numpy array indicating which spins have already been trialed by the algorithm
                
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization"""
    
    old_E = energy_nn(lattice,x)
    lattice[x] *=-1
    dE = energy_nn(lattice,x) - old_E
    E = E + dE
    M = M - lattice[x]*-1 +lattice[x]
    #cluster.append([x,y])
    cluster[x] = True
    if Wolffbacktrack((x+1)%N, cluster, visited):
        E, M = try_add(lattice,(x+1)%N, ClusterSpin,E, M, cluster, visited)
    if Wolffbacktrack((x-1)%N, cluster, visited):
        E, M = try_add(lattice,(x-1)%N, ClusterSpin,E, M, cluster, visited)
    return(lattice,E,M)

def try_add(lattice,x,ClusterSpin,E,M, cluster, visited):
    """Checks of neighbor belongs to cluster, if so then add neighbor to cluster with a certain probability"""
    
    s_help = lattice[x]
    #global tau
    if ((s_help<0) != (ClusterSpin<0)):
        r = np.random.uniform(0,1)
        if r < (1-A_wolff):
            lattice, E, M = grow_cluster(lattice, x, ClusterSpin, E, M, cluster, visited)
        else:
            visited[x] = True
    return(E,M)

def Wolff(lattice, E, M):
    ''' Implements the wolff algorithm for a single timestep.
        
        Parameters: 
            lattice: dtype numpy array
                [N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    x = np.random.randint(0,N)
    cluster = np.full((N), False, dtype=bool)
    visited = np.full((N), False, dtype=bool)
    clusterspin = lattice[x]*-1
    lattice, E, M = grow_cluster(lattice,x,clusterspin, E, M, cluster, visited)
    return(lattice,E,M)

# Correspondence function definitions

In [8]:
def create_frozen_map(lat):
    """ Creates a boolean map of (freezes) all bonds (nn interactions) in the lattice
    
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
        Returns:
            f_map: dtype numpy array
                [N] boolean array containing True if bond is present or False if not. For example spin lattice [1, -1, 1, 1]
                corresponds to f_map [True, True, False, False]
    """
    lat = np.array(lat)
    a = lat[1:]-lat[0:len(lat)-1]; b = lat[0]-lat[len(lat)-1]
    c = np.append(a,b)
    f_map = (c == 0)
    return(f_map)

In [9]:
def backtrack(x,vis,fm,cl):
    """ Part of the cluster decomposition. It recursively checks whether neighbouring spin should be added to the cluster
    """
    if not vis[x]:
        vis[x] = True
        cl.append(x)
        if fm[x]:
            vis, cl = backtrack((x+1)%len(fm),vis,fm,cl)
        if fm[(x-1)%len(fm)]:
            vis, cl = backtrack((x-1)%len(fm),vis,fm,cl)
    return(vis,cl)

In [10]:
def cluster_decomp(lat):
    """ Decomposes the spin lattice into clusters, i.e. creates list of cluster arrays. A cluster array contains the indices of 
        the spins in a cluster
        
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
                
        Returns:
            cls: dtype list
                list of cluster arrays. Each cluster array contains the indices of the spins in that cluster
    """
    visited = np.full((len(lat)), False)
    frozen_map = create_frozen_map(lat)
    #zeros = np.where(np.array(lat)==0)[0]
    #tried = dict.fromkeys(zeros,False)
    cls = []
    for i in range(len(lat)):
        clu = []
        visited, clu = backtrack(i,visited,frozen_map,clu)
        if len(clu) > 0:
            cls.append(clu)
    return(cls)

In [11]:
def create_boundary_graph(lat):
    """ Create boundary graph (collection of nodes) corresponding to the spin lattice
    
        Input: 
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
        Returns: 
            bdGr: dtype networkx graph
                boundary graph 
    """
    n_nodes = len(lat)
    bdGr = nx.Graph()
    for i in range(N):
        bdGr.add_node(i)
    return(bdGr)

In [12]:
def construct_graph(lat,bdGr):
    """ Construct graph from spin lattice according to the AoSD procedure
        
        Input: 
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
            bdGr: dtype networkx graph
                graph containing the boundary nodes (i.e. as constructed by the function 'create_boundary_graph')
            
        Returns: 
            Gr: dtype networkx graph
                AoSD graph 
    """
    lats = []; lats.append(lat)
    shell = 0; n_nodes_shell = len(np.array(lats[shell])); add = 0
    Gr = bdGr.copy()
    CGP_step1 = True
    n_col = 0; n_col_norm = 0
    while(n_nodes_shell>1):
        new_lat = []
        if CGP_step1:
            # perform step 1 of the coarse grain procedure (create domain branches)
            clusters = cluster_decomp(lats[shell])
            # go through clusters
            for i in range(len(clusters)):
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                for j in clusters[i]:
                    value += lats[shell][j]
                    Gr.add_edge(n_Gr,j+add)
                new_lat.append(value)
            CGP_step1 = False
        else:
            # peform step 2 of the coarse grain procedure (average over domains)
            for k in np.arange(0,n_nodes_shell,2):
                n_col_norm += 1
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                #print('n_Gr', n_Gr)
                #print('k',k+add)
                value = lats[shell][k]+lats[shell][k+1]
                if value == 0:
                    value = np.random.choice([-1,1])
                    n_col += 1
                else:
                    value = value/abs(value)
                Gr.add_edge(n_Gr,k+add); Gr.add_edge(n_Gr,k+1+add)
                new_lat.append(value)
            CGP_step1 = True
        lats.append(new_lat)
        shell += 1
        add += n_nodes_shell; n_nodes_shell = len(np.array(lats[shell]))
    return(Gr, n_col, n_col_norm)

# Exhibition

In [13]:
def exhibition(temp, save):
    t_sim = 10
    global T
    T = temp
    #global exp
    #exp = np.array([np.exp(-4/T),1,np.exp(4/T)]) # Store possible values of the acceptance probability (only need first value for metropolis)
    global A_wolff
    A_wolff = np.exp(-2/T) # Needed for acceptance value for Wolff
    
    # Initialize lattice
    lattice = -1*np.ones(N) 
    #lattice = np.random.choice([1,-1],size=(N))

    # Initialize figure
    %matplotlib notebook
    fig = plt.figure(figsize=(10,5)); plt.axis('off'); plt.title('N=%d, T=%.1f'%(N,temp))
    #cur_axes = plt.gca()#cur_axes.axes.get_xaxis().set_visible(False)#cur_axes.axes.get_yaxis().set_visible(False)
    #for spine in plt.gca().spines.values():#   spine.set_visible(False)#plt.figtext(0.9, 0.05, '$x$')#fig.text(0.5,-0.5,'S')#plt.title('N = %d, T = %d'%(temp,N))
    ax1 = fig.add_subplot(1,2,1); ax2 = fig.add_subplot(1,2,2)
    ax1.set_axis_off()

    # Initialize boundary graph
    bdG = create_boundary_graph(lattice)
        
    # Calculate energy and magnetisation by performing a sweep over the whole lattice (only once, as at each step it is updated)
    E = energy_calc(lattice); M = np.sum(lattice) 

    for j in range(t_sim):
        # Perform one MCS using metropolis algorithm:
        #for k in range(N):
            #lattice, E, M = metropolis(lattice,E,M)
            
        # Perform cluster flip using wolff algorithm:
        lattice, E, M = Wolff(lattice,E,M)     
        
        # Draw boundary graph
        nx.draw_networkx(bdG,pos=nx.circular_layout(bdG),with_labels=True,node_color=lattice,ax=ax1,font_size=10)#,cmap=plt.cm.GnBu,vmin=-2,vmax=2)
        
        # Draw bulk graph
        ax2.clear(); ax2.set_axis_off()
        graph, col, col_norm = construct_graph(lattice,bdG)
        pos=graphviz_layout(graph,prog='twopi')
        nx.draw_networkx(graph,pos=pos,with_labels=True,ax=ax2,node_size=20,font_size=8, node_color='r')
    
        fig.canvas.draw()
        #time.sleep(0.1)
        
    if save:
        plt.savefig('1DcorrespondenceN=%dT=%d'%(N,temp))
    #nx.write_gml(graph,'howtree.gml')

In [14]:
N = 50
wg.interact_manual(exhibition, temp=(0.1,10), save=False)

interactive(children=(FloatSlider(value=5.05, description='temp', max=10.0, min=0.1), Checkbox(value=False, de…

<function __main__.exhibition(temp, save)>